### Set Globals

Set the endpoint URL. This can be your loopback address or a remote URL (e.g. AWS load balancer, API Gateway, etc. ) 

In [2]:
!pip install -qU httpx --progress-bar off


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import httpx

CONFIG_API = 'https://<An S3 HTTP accessible bucket>/config.json'

def get_config() -> dict:
    """Method to download client app configuration."""
    response = httpx.get(CONFIG_API)
    print(f'Config:{response.text}')
    props = response.json()['config'] 
    print(f'{props=}')
    return props

props = get_config()

api = props['api'] #SET API Gateway URL here. The code below assumes the URL end in "/".

### Health Check Request

Test GET request on endpoint.

In [3]:
%%time 

url = api 
response = httpx.get(url)
print(f'{response.status_code=}')
print(f'{response.text=}')

response.status_code=200
response.text='{"message":"The GPS Tracker container is up."}'
CPU times: total: 391 ms
Wall time: 598 ms


###  Create User

The code below tests the create user endpoint.

In [4]:
%%time 

response = httpx.post(api + 'users/create_user', json={
  "user_name": "mari",
  "password": "a_password",
  "role": "user"
})
response

CPU times: total: 359 ms
Wall time: 755 ms


<Response [201 Created]>

### Request JSON Web Tokens 

An admin user is created by the ECS task a startup. Code below gets a JWT to be used for subsequent requests.

In [ ]:
response = httpx.post(api + 'auth/token', data={"username": "admin", 
                                                 "password": "a_password_", "grant_type": "password"},
                           headers={"content-type": "application/x-www-form-urlencoded"})
print(f'{response.status_code}')
admin_token = response.json()
print(admin_token)
admin_headers = {"Authorization": f"Bearer {admin_token['access_token']}", "Content-Type": "application/json"}

Now lets sign-in with the user created above and get a JWT.

In [5]:
%%time

response = httpx.post(api + 'auth/token', data={"username": "a_user", 
                                                 "password": "a_password", "grant_type": "password"},
                           headers={"content-type": "application/x-www-form-urlencoded"})
print(f'{response.status_code}')
user_token = response.json()
user_token
user_headers = {"Authorization": f"Bearer {user_token['access_token']}", "Content-Type": "application/json"}

200
CPU times: total: 219 ms
Wall time: 465 ms


###  Get All Users

Verify only admins can call this endpoint.

In [6]:
%%time
response = httpx.get(api + 'users/read_all', headers=admin_headers, timeout=5)
print(response.status_code)
response.json()

200
CPU times: total: 359 ms
Wall time: 1.01 s


[{'id': 1, 'user_name': 'admin', 'role': 'admin'},
 {'id': 2, 'user_name': 'mari', 'role': 'user'}]

Get most recent position updates.

In [7]:
%%time
response = httpx.get(api + 'location/get_latest_locations', headers=admin_headers, timeout=5)
print(response.status_code)
response.json()

200
CPU times: total: 312 ms
Wall time: 560 ms


[{'id': 2, 'user_name': 'mari', 'lat': '40.67423504', 'lng': '-73.97354484'}]

Get all locations for a user.

In [8]:
%%time
response = httpx.get(api + 'location/get_locations', headers=admin_headers, timeout=5)
print(response.status_code)
all_location_updates = response.json()
print(f'Updates: {len(all_location_updates)}')
print(f'Last Update: {all_location_updates[-1]}')

200
Updates: 149
Last Update: {'id': 1, 'user_name': 'admin', 'lat': '40.67431210', 'lng': '-73.97355700'}
CPU times: total: 375 ms
Wall time: 1.08 s


### Web Socket Testing

For WebSocket example client code, see the `ws_client.py` code.